In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())


print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
2
0
NVIDIA GeForce RTX 3090


In [4]:
# iv: image option
length = 440
channel = 128
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [8]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/mountHDD1/LanxHuyen/CVPR2017"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
# block_splits_all, block_splits_single, eeg_14_70, eeg_55_95, eeg_5_95, eeg_raw = os.listdir(torch_models_dir)
print(os.listdir(torch_models_dir))

['imagenet_full', 'eeg_heatmaps_55_95.pth', 'block_splits_by_image_all.pth', 'eeg_heatmaps_14_70.pth', 'splits_by_subject.pth', 'eeg_raw.pth', 'splits_by_subject_1.pth', 'splits_by_image_removed_33.pth', 'imagenet_augmented', 'eeg_55_95_std_downsampled_128Hz_resample.pth', 'eeg_14_70_std.pth', 'imagenet_noeeg_40.zip', 'splits_by_subject_2.pth', 'eeg_55_95_std_downsampled_128Hz.pth', 'eeg_55_95_std.pth', 'eeg_heatmaps_5_95.pth', 'eeg_5_95_std.pth', 'imagenet_full_40_semantic_cls']


In [10]:
eeg_dataset = '/media/mountHDD1/LanxHuyen/CVPR2017/eeg_5_95_std.pth'
splits_all_path = '/media/mountHDD1/LanxHuyen/CVPR2017/block_splits_by_image_all.pth'
# splits_single_path = os.path.join(torch_models_dir, block_splits_single)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
print(eeg_dataset,'\n', splits_all_path, '\n')

/media/mountHDD1/LanxHuyen/CVPR2017/eeg_5_95_std.pth 
 /media/mountHDD1/LanxHuyen/CVPR2017/block_splits_by_image_all.pth 



In [11]:
splits_all = torch.load(splits_all_path)
# splits_single = torch.load(splits_single_path)

In [12]:
print(len(splits_all['splits']))
print(len(splits_all['splits'][0]))

print(len(splits_all['splits'][5]['train']))
print(len(splits_all['splits'][5]['val']))
print(len(splits_all['splits'][5]['test']))
print(splits_all['splits'][0]['train'][:40])
print(splits_all['splits'][1]['train'][:40])
print(splits_all['splits'][2]['train'][:10])
print(splits_all['splits'][3]['train'][:10])


6
3
7984
1996
1985
[0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 27, 29, 33, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 51, 52, 54, 55]
[1, 2, 3, 4, 6, 8, 9, 12, 13, 20, 25, 26, 27, 28, 30, 32, 33, 35, 37, 38, 39, 40, 44, 45, 46, 50, 52, 54, 56, 58, 59, 60, 62, 65, 68, 72, 73, 74, 76, 81]
[2, 3, 4, 5, 6, 7, 8, 10, 11, 13]
[1, 2, 4, 7, 9, 10, 12, 13, 14, 15]


In [13]:
# print(splits_single)
# print(len(splits_single['splits'][0]['train']))
# print(len(splits_single['splits'][0]['val']))
# print(len(splits_single['splits'][0]['test']))

In [14]:
eeg_loaded = torch.load(eeg_dataset)

In [15]:
print(len(eeg_loaded))
print(eeg_loaded.keys())
dataset, labels, images = [eeg_loaded[k] for k in eeg_loaded.keys()]
print(len(labels))
print(len(images))
print(len(dataset))

print(labels)
print(images[0])
print(dataset[0]['eeg'].shape)
print(dataset[0])

3
dict_keys(['dataset', 'labels', 'images'])
40
1996
11965
['n02389026', 'n03888257', 'n03584829', 'n02607072', 'n03297495', 'n03063599', 'n03792782', 'n04086273', 'n02510455', 'n11939491', 'n02951358', 'n02281787', 'n02106662', 'n04120489', 'n03590841', 'n02992529', 'n03445777', 'n03180011', 'n02906734', 'n07873807', 'n03773504', 'n02492035', 'n03982430', 'n03709823', 'n03100240', 'n03376595', 'n03877472', 'n03775071', 'n03272010', 'n04069434', 'n03452741', 'n03792972', 'n07753592', 'n13054560', 'n03197337', 'n02504458', 'n02690373', 'n03272562', 'n04044716', 'n02124075']
n02951358_31190
torch.Size([128, 500])
{'eeg': tensor([[-0.0098,  0.0195,  0.0620,  ...,  0.0638,  0.0120, -0.0118],
        [-0.0045,  0.1303,  0.2673,  ...,  0.0894,  0.0342, -0.0082],
        [ 0.0215, -0.2017, -0.4305,  ..., -0.2022, -0.0940,  0.0188],
        ...,
        [ 0.0160,  0.0707,  0.1005,  ...,  0.2066,  0.1156,  0.0036],
        [-0.0046, -0.0084, -0.0119,  ...,  0.0007, -0.0026, -0.0053],
        [ 

In [16]:
opt = {
    # Dataset options
#     "iv": "image",
#     "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "time_low": 20,
    "time_high": 460,
#     "run": "none",
    "eeg_dataset": eeg_dataset,
    "model_type": "model10",
    "splits_path": splits_all_path,
    "split_num": 0,
    "split_name": "train",
#     "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"from-scratch",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.time_low)

20


In [26]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
# from data_loader import EEGDataset, Splitter, SplitterWithData
from data_loader_CVPR2017 import EEGDataset, Splitter
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
def load_dataset(
#              offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-5) - 6 fold cross validation
             split_name,
#              total, 
#              classes,
#              classifier,
             batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             kind="from-scratch"):        
    # Load dataset
    dataset = EEGDataset(opt, eeg_dataset)
    print("DONE: LOAD DATASET")
#     # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
#     if kind=="from-scratch":
#         relabel = False
#     if kind=="incremental":
#         relabel = False
#     if kind=="no-model-file":
#         relabel = True
    splitter = {split: Splitter(dataset,
                    splits_path,
                    split_num,
                    split_name = split) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [19]:
# Options
opt.classifier = "Stacked_BiLSTM"
opt.batch_size = 64
# opt.kind = "from-scratch"
# opt.run = "imagenet40-1000"
# opt.fold = 5

In [20]:
dataset, loaders, splitter = load_dataset(
#              offset,
             opt.eeg_dataset,
             opt.splits_path,
             opt.split_num, # (0-5) - 6 fold cross validation
             opt.split_name,
#              total, 
#              classes,
#              classifier,
             opt.batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [21]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 8000 idx, val: 2000 idx, test: 2000 idx}
# Each loader batch has {train: 2000 idx, val: 250 idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
for i, (input, target) in enumerate(loaders["train"]):
    if i<20:
        print(f"{i+1}: Target size: {target.size()}; input size: {input.size()}")
# for i in range(0, 40):
#     eeg, label_val = splitter["val"][i]
#     eeg, label_train = splitter["train"][i]
#     print(f"{i+1}: Label val: {label_val}; label train: {label_train}")


<class 'data_loader_CVPR2017.EEGDataset'>
<class 'dict'>
3 [125, 32, 32]
1: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
2: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
3: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
4: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
5: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
6: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
7: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
8: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
9: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
10: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
11: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
12: Target size: torch.Size([64]); input size: torch.Size([64, 128, 440])
13: Target size: torch.Size([64]); input size: t

In [22]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1,128, 440))

DONE: CREATE TORCH CLASSIFIER
classifier_Stacked_BiLSTM(
  (stacked_bilstm): LSTM(128, 128, num_layers=2, batch_first=True, bidirectional=True)
  (output1): Linear(in_features=256, out_features=128, bias=True)
  (relu): ReLU()
  (output2): Linear(in_features=128, out_features=40, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_Stacked_BiLSTM                [1, 40]                   --
├─LSTM: 1-1                              [1, 440, 256]             659,456
├─Linear: 1-2                            [1, 128]                  32,896
├─ReLU: 1-3                              [1, 128]                  --
├─Linear: 1-4                            [1, 40]                   5,160
Total params: 697,512
Trainable params: 697,512
Non-trainable params: 0
Total mult-adds (M): 290.20
Input size (MB): 0.23
Forward/backward pass size (MB): 0.90
Params size (MB): 2.79
Estimated Total Size (MB): 3.92

In [23]:
model_path = (   opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel) )
print(model_path)

Stacked_BiLSTM-440-128


In [24]:
print(opt)

Namespace(results_file='results.pkl', subject=0, time_low=20, time_high=460, eeg_dataset='/media/mountHDD1/LanxHuyen/CVPR2017/eeg_5_95_std.pth', model_type='model10', splits_path='/media/mountHDD1/LanxHuyen/CVPR2017/block_splits_by_image_all.pth', split_num=0, split_name='train', batch_size=64, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='Stacked_BiLSTM')


In [27]:
channel_idx=None
non_classes=None
if opt.kind=="from-scratch":
    loss_history, accuracy_val, accuracy_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

Epoch 1
Train Batch 100 (every 100 batch): Loss=3.6134; accuracy=0.0330
Epoch 1 summary: train_loss: 3.6531 | train_acc: 0.0347 | val_loss: 3.5278 | val_acc: 0.0358
Epoch 2
Train Batch 100 (every 100 batch): Loss=3.3174; accuracy=0.0614
Epoch 2 summary: train_loss: 3.3159 | train_acc: 0.0647 | val_loss: 3.2483 | val_acc: 0.0684
Epoch 3
Train Batch 100 (every 100 batch): Loss=2.8873; accuracy=0.0966
Epoch 3 summary: train_loss: 3.0271 | train_acc: 0.0964 | val_loss: 2.9716 | val_acc: 0.0951
Epoch 4
Train Batch 100 (every 100 batch): Loss=2.7735; accuracy=0.1252
Epoch 4 summary: train_loss: 2.8564 | train_acc: 0.1226 | val_loss: 2.9638 | val_acc: 0.0925
Epoch 5
Train Batch 100 (every 100 batch): Loss=2.7719; accuracy=0.1517
Epoch 5 summary: train_loss: 2.6555 | train_acc: 0.1542 | val_loss: 2.6680 | val_acc: 0.1298
Epoch 6
Train Batch 100 (every 100 batch): Loss=2.6258; accuracy=0.1642
Epoch 6 summary: train_loss: 2.5631 | train_acc: 0.1650 | val_loss: 2.6955 | val_acc: 0.1269
Epoch 7
Tr

ValueError: too many values to unpack (expected 3)

In [ ]:
# val =accuracy_val
# test = accuracy_test

# print("Validation accuracy: ", val)
# print("Test accuracy: ", test)

In [ ]:
print(accuracy_val)